## App prediction server
I have to use notebook because I use remote kernel, both lsa and lda model totals over 4GB and downloading them wouldn't be fun.

In [1]:
from flask import Flask, request
from scipy.sparse import hstack
import pickle
import string
import re
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import nltk
import joblib

nltk.download("stopwords")
nltk.download("punkt")


[nltk_data] Downloading package stopwords to /home/amogus/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/amogus/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
def preprocess(text, stopword_set, stemmer):
    cleaned_text = text.translate(
        str.maketrans(
            "!\#$%&'()*+,<=>?@[]^`{|}~\xa0",
            " " * 26,
        )
    )
    cleaned_text = cleaned_text.lower()
    cleaned_text = re.sub(r"\s+", " ", cleaned_text)
    cleaned_text = re.sub(r"\b\w*_\w+\b", "_term_with_underscore", cleaned_text)
    cleaned_text = re.sub(r"\b\w*-\w+\b", "_term_with_dash", cleaned_text)
    cleaned_text = re.sub(
        r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)",
        "_weburl",
        cleaned_text,
    )
    cleaned_text = re.sub(r"\b[\w\\/]+?[\\/][\w\d_-]+?\b", "_path", cleaned_text)
    cleaned_text = re.sub(r"\b0[xb]?[a-f0-9]+\b", "_non_decimal_number", cleaned_text)
    cleaned_text = re.sub(
        r"\b\d+(px)?\s?[x*]\s?\d+(px)?\b", "_resolution", cleaned_text
    )
    cleaned_text = re.sub(r"\bv\d+(?:\.\d+)?\b", "_version", cleaned_text)
    cleaned_text = re.sub(
        r"\b\d+(?:\.\d+)?\b",
        "_number",
        cleaned_text,
    )
    cleaned_text = re.sub(r"\b(?!_)\w{15,}\b", "_very_long_term", cleaned_text)

    tokenized_text = word_tokenize(cleaned_text)

    sw_removed_text = [word for word in tokenized_text if word not in stopword_set]
    sw_removed_text = [word for word in sw_removed_text if len(word) > 1]
    stemmed_text = " ".join([stemmer.stem(w) for w in sw_removed_text])
    return stemmed_text


app = Flask(__name__)
app.tfidf_vectorizer = joblib.load("resources/07/tfidf_model.joblib")
app.count_vectorizer = joblib.load("resources/07/count_vectorizer.joblib")
app.lsa = joblib.load("resources/07/lsa_model.joblib")
app.lda = joblib.load("resources/07/lda_model.joblib")
app.lgbm = joblib.load("resources/07/gbm_model.joblib")
app.stopword_set = set(stopwords.words())
app.stemmer = PorterStemmer()


@app.route("/predict", methods=["GET"])
def predict():
    response_object = {"status": "success"}
    argList = request.args.to_dict(flat=False)
    title = argList["title"][0]
    body = argList["body"][0]
    count = app.count_vectorizer.transform(
        [preprocess(" ".join([title, body]), app.stopword_set, app.stemmer)]
    )
    tf_idf = app.tfidf_vectorizer.transform(
        [preprocess(" ".join([title, body]), app.stopword_set, app.stemmer)]
    )
    lda = app.lda.transform(count)
    lsa = app.lsa.transform(tf_idf)
    predict = app.lgbm.predict_proba(hstack([tf_idf, lda, lsa]))
    print(f"{predict=}")
    response_object["predict_as"] = "bug" if predict[0][1] > 0.5 else "not bug"
    response_object["bug_prob"] = predict[0][1]
    return response_object


In [3]:
app.run(debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
/opt/conda/envs/ir/lib/python3.10/site-packages/lightgbm/basic.py:1192: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')
127.0.0.1 - - [10/Mar/2024 13:37:44] "GET /predict?title=implement%20new%20styleguid&body=new%20styleguid%20found%20_weburl%20one%20creat%20account%20also%20possibl%20copi%20css%20stuff%20good%20luck HTTP/1.1" 200 -


predict=array([[0.92861345, 0.07138655]])


127.0.0.1 - - [10/Mar/2024 13:37:45] "GET /predict?title=add%20_very_long_term%20gift%20event&body=new%20gifte%20creat HTTP/1.1" 200 -


predict=array([[0.8957689, 0.1042311]])


...

predict=array([[0.62405651, 0.37594349]])


127.0.0.1 - - [10/Mar/2024 13:49:19] "GET /predict?title=dogsonwalk%20alpha%20workout%20summari&body=collect%20view%20workout%20summari%20fade%20due%20alpha%20chang%20_number%20alpha HTTP/1.1" 200 -


predict=array([[0.38617596, 0.61382404]])


127.0.0.1 - - [10/Mar/2024 13:49:19] "GET /predict?title=_term_with_dash%20see%20devic%20unless%20chang%20udev%20rule&body=instal%20udev%20rule%20file%20anoth%20project%20/_path/rules.d/%20_weburl%20_term_with_dash%20could%20see%20eleaf%20aster%20thank HTTP/1.1" 200 -


predict=array([[0.42700604, 0.57299396]])
